In [ ]:
import pandas as pd
import os

In [ ]:
os.getcwd()

'c:\\Users\\John\\Desktop\\Hackathon 2024\\Hackathon Data'

In [ ]:
player = pd.read_csv('player-data/player-combined-data.csv')

In [ ]:
new_columns= player['Position'].str.split(',', expand=True)
player = player.drop(columns = ['Position'])
new_columns.columns = [f'Position_{i}' for i in range(new_columns.shape[1])]  # Modify based on the expected number of splits
player_2 = player.join(new_columns)

In [ ]:
player_3 = player_2.melt(id_vars=player_2.columns.difference(new_columns.columns).tolist(), 
                     value_vars=new_columns.columns, 
                     value_name='Position').drop(columns=['variable'])
player_3 = player_3.dropna(subset=['Position'])

role_metrics = {
    "Forward": ["Goals", "Assists", "Fitness", "Pass Accuracy"],
    "Midfielder": ["Assists", "Pass Accuracy", "Tackles", "Fitness"],
    "Defender": ["Tackles", "Pass Accuracy", "Fitness"],
    "Goalkeeper": ["Saves", "Fitness", "Pass Accuracy"]
}

In [ ]:
# Mapping for position simplification
position_mapping = {
    'CF': 'CF', 'LW': 'LW', 'RW': 'RW', 'RWF': 'RW', 'LWF': 'LW', 
    'GK': 'GK', 'CB': 'CB', 'LCB': 'CB', 'RCB': 'CB', 'LCB3': 'CB', 'RCB3': 'CB',
    'LB': 'LB', 'RB': 'RB', 'LWB': 'LB', 'RWB': 'RB', 'LB5': 'LB', 'RB5': 'RB',
    'AMF': 'AMF', 'LAMF': 'AMF', 'RAMF': 'AMF',
    'CMF': 'CMF', 'LCMF': 'CMF', 'RCMF': 'CMF', 'LCMF3': 'CMF', 'RCMF3': 'CMF',
    'DMF': 'DMF', 'LDMF': 'DMF', 'RDMF': 'DMF',
    '0': '',  # Handle positions like '0' with an explicit category
}

position_mapping_2 = {
    'CF': 'Forward', 'LW': 'Forward', 'RW': 'Forward', 'CMF': 'Midfield',
    'AMF': 'Midfield', 'DMF': 'Midfield', 'LB': 'Back', 'RB': 'Back', 'CB': 'Back',
    'GK': 'Goalkeeper'
}

# Function to apply the mapping
def simplify_positions(position):
    position = position.strip()  # Remove leading/trailing spaces
    return position_mapping.get(position, position)  # Default to original if no mapping

def simplify_positions_2(position):
    position = position.strip()  # Remove leading/trailing spaces
    return position_mapping_2.get(position, position)  # Default to original if no mapping


# Apply simplification to your dataset
player_3['Simple_Position'] = player_3['Position'].apply(simplify_positions)
player_3['Simple_Position'] = player_3['Simple_Position'].apply(simplify_positions_2)
# player_3['Position'] = player_3['Position'].replace(r'\d+', '', regex=True).str.strip()
player_3['Simple_Position'] = player_3['Simple_Position'].replace('', pd.NA)

# Group by 'Name' and calculate the most common position for each player
most_common_position = player_3.groupby('player_name')['Simple_Position'].apply(lambda x: x.mode()[0] if not x.mode().empty else 'Unknown')

# Fill missing positions with the most common position for each player
player_3['Simple_Position'] = player_3.apply(lambda row: most_common_position[row['player_name']] if pd.isnull(row['Simple_Position'])  else row['Simple_Position'], axis=1)

# Count occurrences of each simplified position
simplified_counts = player_3['Simple_Position'].value_counts()

In [ ]:
player_3.to_csv('player-data/cleaned_player.csv', index = False)